In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 2.4 MB/s 
     |████████████████████████████████| 3.1 MB 34.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=2cb842e05dd096f17e8f5a162cb9d295810cb7b785f1cb18e4f6178451d95574
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers import losses
from sentence_transformers import datasets
from sentence_transformers import InputExample
import transformers
print(transformers.__version__)

4.12.2


In [ ]:
word_emb = models.Transformer('sentence-transformers/msmarco-bert-base-dot-v5')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules = [word_emb, pooling])

Downloading:   0%|          | 0.00/636 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

We use <b>MultipleNegativesRankingLoss</b> as our loss function for fine-tuning as we only have positive pairs in our dataset <br>(e.g. question->answer pairs)

In [ ]:
# train_df = pd.read_csv("../0.Datasets/train_test_split/train.csv")
train_df = pd.read_csv("./train.csv")

train_df.head()
train_df.shape

(13657, 5)

In [ ]:
fine_tune_examples = []
for _,row in train_df.iterrows():
    question = row["question"]
    answer = row["answer"]
    fine_tune_examples.append(InputExample(texts=[question,answer]))

len(fine_tune_examples)

13657

In [ ]:
train_dataloader = datasets.NoDuplicatesDataLoader(fine_tune_examples, batch_size=8)

In [ ]:
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
len(fine_tune_examples)

13657

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1707 [00:00<?, ?it/s]

In [ ]:
model.save('./finetuned-bertbase-1epoch')

In [ ]:
!zip -r ./finetuned-bertbase-1epoch.zip ./finetuned-bertbase-1epoch

  adding: finetuned-bertbase-1epoch/ (stored 0%)
  adding: finetuned-bertbase-1epoch/tokenizer_config.json (deflated 41%)
  adding: finetuned-bertbase-1epoch/pytorch_model.bin (deflated 7%)
  adding: finetuned-bertbase-1epoch/tokenizer.json (deflated 59%)
  adding: finetuned-bertbase-1epoch/config_sentence_transformers.json (deflated 25%)
  adding: finetuned-bertbase-1epoch/sentence_bert_config.json (deflated 4%)
  adding: finetuned-bertbase-1epoch/modules.json (deflated 53%)
  adding: finetuned-bertbase-1epoch/README.md (deflated 58%)
  adding: finetuned-bertbase-1epoch/special_tokens_map.json (deflated 40%)
  adding: finetuned-bertbase-1epoch/vocab.txt (deflated 53%)
  adding: finetuned-bertbase-1epoch/config.json (deflated 48%)
  adding: finetuned-bertbase-1epoch/1_Pooling/ (stored 0%)
  adding: finetuned-bertbase-1epoch/1_Pooling/config.json (deflated 47%)


In [ ]:
from google.colab import files
files.download("./finetuned-bertbase-1epoch.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>